In [1]:
#!pip install -Uqqq pip
#!pip install -qqq bitsandbytes==0.39.0
#!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.30.0.dev0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --upgrade transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 69.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.0.dev0
    Uninstalling transformers-4.30.0.dev0:
      Successfully uninstalled transformers-4.30.0.dev0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.0.dev0
    Uninstalling accelerate-0.20.0.dev0:
      Successfully uninstalled accelerate-0.20.0.dev0


In [3]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.8 MB/s eta 0:00:00


In [4]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
!pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 96.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187815463 sha256=d944fc7d2f962bce83fc4708c2fc0c21eaf8255962a0b350ae919362a51b7ef2
  Stored in directory: /root/.cache/pip/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built flash_attn


In [6]:
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

In [7]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [8]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [9]:
config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)


trainable params: 83886080 || all params: 4624486400 || trainables%: 1.813954518279046


In [10]:
notebook_login()

In [11]:
prompt = """
<human>: JAVASCRIPT TEST: When I am on the https://www.example.com page then the page should have 'Example Domain' in the title
<assistant>:
""".strip()

In [12]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.7,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model.generation_config = generation_config


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [13]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: JAVASCRIPT TEST: When I am on the https://www.example.com page then the page should have 'Example Domain' in the title
<assistant>: I'll create a test using Selenium that checks the page title when visiting https://www.example.com.
<assistant>: I'll write the test in JavaScript using the Selenium WebDriver.
<assistant>: The test will open the browser, navigate to the specified URL, check the title, and then close the browser.
<assistant>: I'll make sure the test is efficient and doesn't cause any issues on the actual website.
<assistant>: I'll also include error handling to ensure the test runs smoothly.
</human>: Okay, so I need to write a test using Selenium in JavaScript that checks if the page title is 'Example Domain' when I'm on https://www.example.com.
</human>: Alright, let me think about how to approach this.
</human>: First, I need to set up the Selenium environment. I know that I'll need a WebDriver, probably using a browser like Chrome.
</human>: I should make sure

In [14]:
from datasets import Dataset
Dataset.cleanup_cache_files

<function datasets.arrow_dataset.Dataset.cleanup_cache_files(self) -> int>

In [36]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("/content/train_dataset.csv")

# Convert DataFrame into a Hugging Face Dataset
dataset = Dataset.from_pandas(df[['Prompts', 'Output']].rename(columns={'Prompts': 'instruction', 'Output': 'output'}))

# Function to format text
def generate_prompt(example):
    instruction = example["instruction"]
    output = example["output"]

    return {"text": f"<human>: {instruction}\n<assistant>: {output}"}

# Apply the prompt formatting
dataset = dataset.map(generate_prompt)

# Check dataset columns before tokenization
print("Dataset columns before tokenization:", dataset.column_names)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["instruction", "output", "text"])

tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])

tokenized_dataset.set_format("torch")
print("Dataset columns after tokenization:", tokenized_dataset.column_names)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Dataset columns before tokenization: ['instruction', 'output', 'text']


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Dataset columns after tokenization: ['input_ids', 'attention_mask', 'labels']


/usr/local/lib/python3.11/dist-packages/datasets/table.py:1381: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1407: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [38]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=0.5,
    max_steps=100,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=10,
    output_dir="experiments",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    gradient_checkpointing=True,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# Disable cache to save memory
model.config.use_cache = False

trainer.train()

Step,Training Loss
10,1.886000
20,0.624500
30,0.497800
40,0.398600
50,0.418300
60,0.444200
70,0.428400
80,0.397100
90,0.378600
100,0.373300


TrainOutput(global_step=100, training_loss=0.5846736884117126, metrics={'train_runtime': 568.2069, 'train_samples_per_second': 0.352, 'train_steps_per_second': 0.176, 'total_flos': 4662565326028800.0, 'train_loss': 0.5846736884117126, 'epoch': 0.03333333333333333})

In [39]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "roku02/deepseek-8B"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from transformers import BitsAndBytesConfig
import torch

# Base model
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# BitsAndBytes quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",  
    trust_remote_code=True,
    quantization_config=bnb_config,
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Move model to appropriate device (CUDA or CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

fine_tuned_model = "roku02/deepseek-8B"  
model = PeftModel.from_pretrained(model, fine_tuned_model)

print("Model and tokenizer loaded successfully!")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/336M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:398: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


Model and tokenizer loaded successfully!


In [8]:
tokenizer.save_pretrained("/content/trained-model")

('/content/trained-model/tokenizer_config.json',
 '/content/trained-model/special_tokens_map.json',
 '/content/trained-model/tokenizer.json')

In [11]:
generation_config = model.generation_config
generation_config.max_new_tokens = 350
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [13]:
%%time
device = "cuda:0"

prompt = """
<human>: JAVASCRIPT TEST: When I open the https://www.example.com page, it should display a heading with the text 'Welcome to Example Domain'
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<assistant>: test("should contain 'Welcome to Example Domain' as a heading on the 'https://www.example.com' page", async ({ page }) => {
await page.goto('https://www.example.com');
const headingText = await page.getByRole('heading', { name: /Welcome to Example Domain/i }).textContent();
expect(headingText).toContain('Welcome to Example Domain');
<assistant>: test("should have 'Example Domain' in the title when on the 'https://www.example.com' page", async ({ page }) => {
await page.goto('https://www.example.com');
const pageTitle = await page.title();
expect(pageTitle).toContain('Example Domain');
<assistant>: test("should verify screenshot of the 'https://www.example.com' page", async ({ page }) => {
await page.goto('https://www.example.com');
await page.screenshot({ fullPage: true });
<assistant>: test("should have 'Welcome to Example Domain' as a heading when on the 'https://www.example.com' page", async ({ page }) => {
await page.goto('https://www.example.com');
const headingText =